<a target="_blank" href="https://colab.research.google.com/drive/1142VbFt8kLRz7amDi9UQt5LaJvIcTaZt?usp=sharing">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# CircuitsDNA - Approximate Circuit Synthesis based on Evolutionary Algorithm 

```
Submission to IEEE SSCS Open-Source Ecosystem “Code-a-Chip” Travel Grant Awards at ISSCC'26
SPDX-License-Identifier: GPL-3.0-only
```


|Name|Affiliation|IEEE Member|SSCS Member|
|:--:|:----------:|:----------:|:----------:|
|Ruichen Qi <br /> Email ID: ruichen_qi@brown.edu|Brown University|No|No|
|Junyi Luo <br /> Email ID: junyi_luo@brown.edu|Brown University|No|No|

<br>

---

## 1. Introduction



### 1.1 Background

According to Moore’s Law and Dennard scaling, continuous transistor miniaturization since 1974 has enabled exponential growth in device density—doubling with each generation—while maintaining higher clock speeds under constant power density. However, around 2007, the benefits of Dennard scaling began to fade due to leakage currents and power density constraints, and by 2012, scaling had largely halted.

Modern computing systems now face severe power and thermal bottlenecks. As transistor density continues to rise, heat dissipation has become a fundamental limitation: chips can no longer operate all transistors simultaneously without exceeding safe thermal limits. This results in “dark silicon,” where only a portion of the available cores can remain active to avoid overheating. Elevated temperatures also degrade device reliability, accelerating wear-out mechanisms and shortening system lifetime.

To address these challenges, approximate computing has emerged as a promising solution. By relaxing accuracy requirements in error-tolerant applications, approximate designs can substantially reduce power consumption and heat generation while maintaining acceptable output quality. This paradigm is especially suitable for neural network and large language model (LLM) accelerators, which are inherently resilient to small arithmetic errors. Minor inaccuracies in multiplications or accumulations typically have negligible impact on model accuracy, allowing designers to adopt approximate multipliers, adders, or reduced-precision datapaths for significant savings in power, area, and latency.

Moreover, fine-tuning techniques can further mitigate hardware-induced errors. By retraining or adapting model parameters on the approximate hardware, most of the lost accuracy can be recovered while retaining energy efficiency gains. This makes approximate computing particularly attractive for large-scale AI accelerators, where computational and memory demands are exceptionally high.

### 1.2 Motivation

Despite its promise, approximate computing still faces practical challenges in logic synthesis. Traditional design methods—such as manual simplification or heuristic gate pruning often rely on structural assumptions and struggle to scale for complex arithmetic blocks like multipliers. These methods typically yield locally optimized solutions and lack the flexibility to explore the vast combinational design space.

To overcome these limitations, genetic algorithms (GAs) offer an effective alternative. By evolving populations of candidate circuits through mutation, crossover, and selection, GAs can efficiently explore discrete, non-linear design spaces without requiring gradient information or explicit models. Their ability to support multi-objective optimization makes them ideal for balancing trade-offs among accuracy, power, area, and delay.

However, most GA-based approximate synthesis approaches remain limited to small-scale demonstrations and rarely connect circuit-level optimization to system-level evaluation. This work bridges that gap by introducing an end-to-end GA-driven framework that synthesizes approximate computing circuits and evaluates their real-world impact on neural network tasks such as image classifications

### 1.3 Notebook Overview

This notebook presents a genetic-algorithm–based framework for approximate logic synthesis and its application-level evaluation on deep learning models including Fashion-MNIST (LeNet-5) and CIFAR-100 (ResNet-18/ResNet-20).
The framework supports both random circuit generation and optimization from a seed netlist, providing a complete flow from synthesis to performance analysis.

Workflow summary:

An 8-bit signed multiplier is synthesized and verified using Yosys-ABC, iVerilog, and OpenSTA.

The genetic algorithm performs approximate logic synthesis on the extracted netlist under various error and area constraints.

The evolved approximate design is analyzed in OpenSTA to extract power, timing, and LUT information.

Finally, the approximate multipliers are integrated into deep learning workloads (LeNet-5, ResNet-18, and ResNet-20) to evaluate accuracy–efficiency trade-offs.

[FIgure] Workflow of our algorithm and approx multiplier based systolic array

---

## 2. Setting up of Open Source Tools

The latest versions of Yosys, iVerilog, and OpenSTA should be installed. There are many installation tutorials available online. You can either install them manually by following the instructions on their GitHub pages, or use the script we’ve provided:



In [ ]:
# Import modules
!make create_env
!make check_env

In [ ]:
!python -m venv venv
!venv/bin/python -m pip install --upgrade pip ipykernel
!venv/bin/python -m pip install -r requirements.txt
!venv/bin/python -m ipykernel install --user --name=venv --display-name "Python (venv)"

For the following steps, please switch the python kernal to the virtual environment we just created.

---

## 3. 8-bit Signed Multiplier Synthesis and Verification

In this demonstration, an 8-bit signed multiplier is synthesized and verified based on the GF180 technology as an example. All necessary files for synthesis and verification are prepared. Only six basic gate types — AND, NAND, OR, XOR, XNOR, and INV are used for synthesis, though the framework can be easily extended to include other logic gates or larger building blocks such as full adders (FA), barrel shifters, multipliers, etc. Here, we use these six gate types solely as a representative example.

Goldenbrick generation and behavior simulation:

In [ ]:
# Goldenbrick generation
!make goldenbrick_gen

# Behavior simulation
!make sim_behavior
!make check_behavior

Synthesis, post-synthesis simulation and verification:

In [ ]:
# Synthesis with a target frequency of 100MHz
!make run_synth

# Post-synthesis simulation and verification
!make sim_synth
!make check_synth

# Static Timing Analysis (STA)
!make sta

---
## 4. Approximate Multiplier Synthesis from the Synthesized Netlist via Genetic Algorithm

In this section, we use a genetic algorithm (GA) to evolve approximate 8-bit signed multipliers. Genetic algorithms struggle with large circuits due to the exponentially growing search space, high computational cost, and poor scalability. Therefore, our GA starts from the seed circuit, which is loaded from the synthesized Verilog netlist in previous step.

During each generation, the algorithm evaluates all candidate circuits by simulating their outputs over all input patterns. The fitness function jointly considers circuit accuracy and area efficiency — penalizing individuals with large worst-case errors (WCE > ε_th) while favoring smaller transistor counts. Multiple error metrics such as NMED, ER, WCE, MRE, and sMAPE are recorded for analysis.

New individuals are created through mutation operators, including:

Add / delete node: randomly insert or remove a gate;

Change gate type: switch to another logic primitive;

Rewire inputs or outputs: alter signal connectivity;

Merge equivalent nodes: remove redundant subcircuits.

By iteratively applying mutation, pruning inactive nodes, and selecting the best individuals, the GA searches the discrete, irregular design space to obtain compact approximate multipliers with bounded output error. This process enables automatic approximate logic synthesis directly at the gate level, providing a flexible framework for exploring accuracy–area trade-offs under the GF180 technology library.




[FIgure] Algorithm explaination

Since this algorithm is implemented using Cpp for acceleration, firstly we need to compile the code:

In [ ]:
# Compile the GA code
!make compile_ga

Then we're ready to evolve approximate logics using this algorithm. The evolution may take several hours to finish, depending on the performance of current platforms.

In [ ]:
# Start the GA evolution
!make run_ga

Here we use the evolution with an relative worst case error threshold of 0.5% as an example. All the experiments in this section were run on an AMD Ryzen 9 7845HX laptop for 14 hours.

<img src="images/evolution_process_epsEnd_0.005.png"
     alt="Evolution Process"
     style="width:800px;max-width:100%;border:1px solid #ccc;">

A grid search over the relative worst-case error threshold was performed, and the results are shown below. With only a 0.5% relative worst-case error, the equivalent transistor count was reduced to 76.5% of the original size.

<img src="images/equivalent_transistor_count_vs_epsEnd.png"
     alt="Evolution Process"
     style="width:800px;max-width:100%;border:1px solid #ccc;">

<img src="images/error_metrics_vs_epsEnd.png"
     alt="Evolution Process"
     style="width:800px;max-width:100%;border:1px solid #ccc;">

---
## 5. Analysis and Verification of Generated Approximate Multiplier

In this section, we map the generated netlist to a specific semiconductor technology (GF180, for example) and perform functional verification as well as power, performance, and area (PPA) analysis using OpenSTA. The design is mappd to standard cells from the target technology library, and timing, power, and area reports are extracted to evaluate the quality of the evolved approximate multiplier.



The program would save log files as well as the best evolved netlist file. In the log file you can get the equivelent transistor number, best fitness value and all the error metrics for each generation. The generated netlist file hasn't been mapped to a certain technology. Therefore, we need to map it to GF180 through: 

In [ ]:
# Map the generated netlist to GF180
!make map_netlist

Then we can extract the truth table of the generated approximate multiplier using the following command. In this step the mapped netlist would be instantialized in a testbench written in systemverilog and evaluated by behavior simulation using iVerilog.

In [ ]:
# Extract the truth table of the generated approximate multiplier
!make sim_approx_netlist

We can also check the error metrics from the extracted truth table for verifications

In [ ]:
# Todo: Error metrics verification
!make check_approx_netlist

---
## 6. Implementation of Deep Learning Tasks including Fashion-MNIST on LeNet-5 and CIFAR-100 on ResNet-18/ResNet-20 with Approximate Computing Unit

---
## 7. Discussion


---
## 8. Conclusion


In conclusion, 